In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/08/22 11:29:06 WARN Utils: Your hostname, Grahams-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.169.1.168 instead (on interface en0)
23/08/22 11:29:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/22 11:29:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Read data from AWS S3 into a DataFrame
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
file_path = "file://" + SparkFiles.get("home_sales_revised.csv")

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Create a temporary view of the DataFrame
view_name = "home_sales_view"
df.createOrReplaceTempView(view_name)

In [4]:
# Verify the column names and structure in the DataFrame
print(df.columns)  # Check the list of column names

# Check the schema of the DataFrame
df.printSchema()



['id', 'date', 'date_built', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view']
root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# Run SQL query to calculate the average price for a four-bedroom house sold in each year
query = """
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE bedrooms = 4
    GROUP BY year
    ORDER BY year
"""

# Execute the query and show the result
result = spark.sql(query)
result.show()



+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Query to calculate the average price for homes with 3 bedrooms and 3 bathrooms for each year built
query = """
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
"""

# Execute the query and show the result
result = spark.sql(query)
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Query to calculate the average price for homes with specific criteria
query = """
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
"""

# Execute the query and show the result
result = spark.sql(query)
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE price >= 350000
    GROUP BY view
"""
result = spark.sql(query)
result.show()


start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 7.605552673339844e-05 seconds ---


In [11]:
# Cache the table
spark.catalog.cacheTable("home_sales_view")

# Check if the table is cached
is_cached = spark.table("home_sales_view").is_cached



23/08/22 11:31:15 WARN CacheManager: Asked to cache already cached data.


In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Uncached Query
start_time_uncached = time.time()

# Run the query without using cached data
query_uncached = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE price >= 350000
    GROUP BY view
    ORDER BY avg_price DESC
"""
result_uncached = spark.sql(query_uncached)
result_uncached.show()

uncached_runtime = time.time() - start_time_uncached
print("Uncached Runtime:", uncached_runtime, "seconds")

# Cache the table
spark.catalog.cacheTable("home_sales_view")

# Cached Query
start_time_cached = time.time()

# Run the same query using cached data
query_cached = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_view
    WHERE price >= 350000
    GROUP BY view
    ORDER BY avg_price DESC
"""
result_cached = spark.sql(query_cached)
result_cached.show()

cached_runtime = time.time() - start_time_cached
print("Cached Runtime:", cached_runtime, "seconds")



+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

Uncached Runtime: 0.6558511257171631 seconds
+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

Cached Runtime: 0.1865699291229248 seco

23/08/22 11:32:21 WARN CacheManager: Asked to cache already cached data.


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
# Write the DataFrame to Parquet partitioned by "date_built"
partition_column = "date_built"
output_path = "path_to_output_directory"

df.write.partitionBy(partition_column).parquet(output_path)


In [14]:
# 11. Read the formatted parquet data.

partitioned_data_path = "path_to_output_directory"

partitioned_df = spark.read.parquet(partitioned_data_path)


In [15]:
# 12. Create a temporary table for the parquet data.

view_name = "partitioned_home_sales_view"
partitioned_df.createOrReplaceTempView(view_name)


In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

# Define the query
query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM partitioned_home_sales_view
    WHERE price >= 350000
    GROUP BY view
"""

# Calculate runtime
start_time = time.time()

# Execute the query and show the result
result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.5871067047119141 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales_view")


In [18]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.table("home_sales_view").is_cached
print("Is home_sales_view cached:", is_cached)



Is home_sales_view cached: False
